# Machine learning with basic housing data

Experimenting with different algorithms on a non-geographical and basic geographical feature set.

### TODO:
- maybe add intercept for linear SOOS model

### Import packages

In [1]:
import json
import math
import warnings
warnings.filterwarnings(action="ignore")

from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

from utils import make_train_test, get_metrics, cross_validation, soos_validation

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from IPython.display import display_html

Definde constants.

- ``PATH``: Path to the base data folder
- ``K_FOLDS``: Number of folds to perform for cross validation

In [2]:
PATH = "C:/Users/Tim/.keras/datasets/wikipedia_real_estate/"
K_FOLDS = 5
DUMMIES = []  # e.g. ["MUNICODE"]

Load structured data and print columns.

In [3]:
structured = pd.read_csv(PATH+"structured_preprocessed.csv")
print(structured.shape)
print(structured.columns)
structured.head(10)

(9554, 65)
Index(['_id', 'PROPERTYZIP', 'MUNICODE', 'SCHOOLCODE', 'NEIGHCODE', 'LOTAREA',
       'SALEDATE', 'SALEPRICE', 'FAIRMARKETTOTAL', 'STORIES', 'YEARBLT',
       'GRADE', 'CONDITION', 'CDU', 'TOTALROOMS', 'BEDROOMS', 'FULLBATHS',
       'HALFBATHS', 'FIREPLACES', 'BSMTGARAGE', 'FINISHEDLIVINGAREA',
       'latitude', 'longitude', 'DISTRICT', 'SALEYEAR', 'SALEYEAR_STR_2015',
       'SALEYEAR_STR_2016', 'SALEYEAR_STR_2017', 'SALEYEAR_STR_2018',
       'SALEYEAR_STR_2019', 'SALEYEAR_STR_2020',
       'HEATINGCOOLINGDESC_CentralHeat',
       'HEATINGCOOLINGDESC_CentralHeatwithAC', 'OWNERDESC_REGULAR',
       'OWNERDESC_REGULAR-ETAL', 'OWNERDESC_REGULAR-ETUXORETVIR',
       'STYLEDESC_BI-LEVEL', 'STYLEDESC_BUNGALOW', 'STYLEDESC_CAPECOD',
       'STYLEDESC_COLONIAL', 'STYLEDESC_CONTEMPORARY', 'STYLEDESC_OLDSTYLE',
       'STYLEDESC_RANCH', 'STYLEDESC_SEMIDETACHED', 'STYLEDESC_SPLITLEVEL',
       'STYLEDESC_TUDOR', 'STYLEDESC_VICTORIAN', 'EXTFINISH_DESC_Brick',
       'EXTFINISH_DESC_

,_id,PROPERTYZIP,MUNICODE,SCHOOLCODE,NEIGHCODE,LOTAREA,SALEDATE,SALEPRICE,FAIRMARKETTOTAL,STORIES,...,ROOFDESC_SLATE,ROOFDESC_TILE,BASEMENTDESC_Crawl,BASEMENTDESC_Full,BASEMENTDESC_None,BASEMENTDESC_Part,ROOFDESC_ROLL.1,ROOFDESC_SHINGLE.1,ROOFDESC_SLATE.1,ROOFDESC_TILE.1
0,161705,15122,870,45,87005,10899,05-01-2018,145000.0,76700,1.0,...,0,0,0,1,0,0,0,1,0,0
1,530852,15146,879,18,87905,10691,05-13-2019,139997.0,106200,1.0,...,0,0,0,1,0,0,0,1,0,0
2,144978,15202,826,2,82601,11813,05-26-2017,170000.0,135300,1.0,...,0,0,0,1,0,0,0,1,0,0
3,436602,15202,803,29,80302,5324,06-06-2017,145000.0,117300,2.0,...,0,0,0,1,0,0,0,1,0,0
4,145066,15218,114,47,11403,3600,04-09-2016,325000.0,250000,2.0,...,0,0,0,1,0,0,0,1,0,0
5,145137,15228,926,26,92607,6406,04-30-2015,172900.0,137300,2.0,...,0,0,0,1,0,0,0,1,0,0
6,145246,15241,950,42,95001,38376,12-17-2015,817000.0,751600,2.0,...,0,0,0,1,0,0,0,1,0,0
7,529513,15132,409,23,40005,3844,01-09-2020,39000.0,45100,1.0,...,0,0,0,1,0,0,0,1,0,0
8,146103,15212,127,47,12703,5284,06-30-2016,65000.0,52800,1.5,...,0,0,0,1,0,0,0,1,0,0
9,146155,15212,127,47,12701,5544,11-10-2018,162000.0,111200,1.0,...,0,0,0,1,0,0,0,1,0,0


Make train/test split

In [4]:
X_columns, data_sets, error_df = make_train_test(structured, dummies=DUMMIES)
X, y, X_train, X_test, y_train, y_test, X_train_train, X_train_val, y_train_train, y_train_val = data_sets


(7165, 53): (5373, 54) + (1792, 54)
(7165,): (5373,) + (1792,)
(2389, 53)
(2389,)


In [5]:
results_df = pd.DataFrame()

## Only structured data

### Linear model

In [6]:
model_01 = linear_model.LinearRegression()
# model_01 = linear_model.Lasso()
# model_01 = linear_model.Ridge()
model_01.fit(X_train, y_train)

LinearRegression()

In [7]:
y_pred_01 = model_01.predict(X_test)
metrics_01 = get_metrics(y_test, y_pred_01)

MAE:  43549
RMSE: 60150
MAPE: 27.3%
R^2:  0.789


In [8]:
results_df["Linear: S"], X_01_columns = cross_validation(model_01, X, y, K_FOLDS)

  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  44170
RMSE: 62380
MAPE: 28.17%
R^2:  0.791


### Catboost

In [9]:
model_02 = CatBoostRegressor()
model_02.fit(X=X_train, y=y_train, verbose=False)

In [10]:
y_pred_02 = model_02.predict(X_test)
metrics_02 = get_metrics(y_test, y_pred_02)

MAE:  39610
RMSE: 56633
MAPE: 24.2%
R^2:  0.813


In [11]:
results_df["Catboost: S"], X_02_columns = cross_validation(model_02, X, y, K_FOLDS)

  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  40980
RMSE: 59117
MAPE: 25.53%
R^2:  0.812


Save error df

In [12]:
error_df.to_csv(PATH+"results/errors_basic.csv", index=None)
results_df.index = ["MAE", "RMSE", "MAPE", "R^2"]
results_df.head()

,Linear: S,Catboost: S
MAE,44170.000,40980.000
RMSE,62380.000,59117.000
MAPE,28.170,25.530
R^2,0.791,0.812


## Structured and basic spatial membership

Different types of aggregated membership are tested for their predictive performance.

### Linear model

In [28]:
for spatial in ["PROPERTYZIP", "MUNICODE", "SCHOOLCODE", "NEIGHCODE"]:
    # make dataset with spatial column
    X_columns, data_sets, error_df = make_train_test(structured, dummies=[spatial], verbose=False)
    X, y, X_train, X_test, y_train, y_test, X_train_train, X_train_val, y_train_train, y_train_val = data_sets
    
    # model_spatial = linear_model.LinearRegression()
    model_spatial = linear_model.Lasso()
    # model_spatial = linear_model.Ridge()
    
    print(f"Spatial choice: {spatial}")
    metrics = cross_validation(model_spatial, X, y, K_FOLDS, verbose_drop=False)
    print("")

Spatial choice: PROPERTYZIP


  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  35824
RMSE: 51579
MAPE: 22.27%
R^2:  0.857

Spatial choice: MUNICODE


  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  32881
RMSE: 47805
MAPE: 20.33%
R^2:  0.877

Spatial choice: SCHOOLCODE


  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  38133
RMSE: 55618
MAPE: 24.33%
R^2:  0.834

Spatial choice: NEIGHCODE


  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  30917
RMSE: 45129
MAPE: 18.87%
R^2:  0.891



### CatBoost

In [13]:
for spatial in ["PROPERTYZIP", "MUNICODE", "SCHOOLCODE", "NEIGHCODE"]:
    # make dataset with spatial column
    X_columns, data_sets, error_df = make_train_test(structured, dummies=[spatial], verbose=False)
    X, y, X_train, X_test, y_train, y_test, X_train_train, X_train_val, y_train_train, y_train_val = data_sets
    
    model_spatial = CatBoostRegressor()
    
    print(f"Spatial choice: {spatial}")
    metrics = cross_validation(model_spatial, X, y, K_FOLDS, verbose_drop=False)
    print("")

Spatial choice: PROPERTYZIP


  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  31490
RMSE: 46930
MAPE: 18.67%
R^2:  0.882

Spatial choice: MUNICODE


  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  30505
RMSE: 45284
MAPE: 18.15%
R^2:  0.89

Spatial choice: SCHOOLCODE


  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  32162
RMSE: 49000
MAPE: 19.31%
R^2:  0.871

Spatial choice: NEIGHCODE


  0%|          | 0/5 [00:00<?, ?it/s]


MAE:  32247
RMSE: 46979
MAPE: 19.75%
R^2:  0.882



## Spatial out-of-sample Linear

In [14]:
estimator = linear_model.LinearRegression()
error_df_soos, col_names_linear, avg_fis_linear, metrics = soos_validation(estimator, structured)
maes, rmses, mapes, r_squareds = metrics

Predicting district 1/13
Predicting district 2/13
Predicting district 3/13
Predicting district 4/13
Predicting district 5/13
Predicting district 6/13
Predicting district 7/13
Predicting district 8/13
Predicting district 9/13
Predicting district 10/13
Predicting district 11/13
Predicting district 12/13
Predicting district 13/13

Weighted metrics:
MAE:  45730
RMSE: 62149
MAPE: 28.99%
R^2:  0.536


In [15]:
error_df_soos.to_csv(PATH+"results/errors_soos_basic.csv")
error_df_soos

,id,lat,long,district,prediction,error
0,71623,40.664950,-79.719952,district_7,103477.965565,82497.034435
1,396546,40.450605,-80.217889,district_1,214352.245878,-19352.245878
2,161454,40.477051,-79.777996,district_8,216516.266607,13483.733393
3,461687,40.606085,-79.930633,district_3,545070.228069,49929.771931
4,136057,40.378484,-80.062464,district_5,801805.934620,-203805.934620
...,...,...,...,...,...,...
9549,289011,40.551787,-79.996615,district_1,274294.157015,705.842985
9550,250118,40.451706,-79.854802,district_10,133769.882295,-13769.882295
9551,263392,40.607721,-79.930186,district_3,191060.544102,-96060.544102
9552,459551,40.382627,-80.064891,district_5,306537.515547,-119537.515547


In [16]:
districts = ["district_"+str(i) for i in range(1,14)]
metrics_df = pd.DataFrame(data={"district":districts, "mae":maes, "rmse":rmses, "mapes":mapes, "R^2":r_squareds})
metrics_df = metrics_df.set_index("district")
metrics_df.to_csv(PATH+"results/errors_soos_district_basic.csv")

# add more information about each district to characterize
df_agg = structured.groupby(by="DISTRICT").mean()
df_agg  = df_agg[["SALEPRICE", "LOTAREA", "YEARBLT", "STORIES"]]
metrics_df_agg = pd.concat([metrics_df, df_agg], axis=1)

metrics_df_agg

,mae,rmse,mapes,R^2,SALEPRICE,LOTAREA,YEARBLT,STORIES
district_1,36078.160575,49507.527432,18.560255,0.699025,219888.788503,18311.494577,1959.376356,1.557158
district_2,55901.164345,80907.973640,16.813528,0.747479,347195.029630,28928.464550,1968.440212,1.703175
district_3,47911.737015,69979.062743,20.480843,0.819349,261041.727106,22902.147436,1959.855311,1.500916
district_4,37997.784729,51479.450066,25.665238,0.689832,191448.015873,16155.658009,1956.578644,1.485570
district_5,43495.971226,57947.928416,16.362635,0.774242,279481.883882,13725.758232,1956.907279,1.650347
district_6,33176.442052,43469.869970,25.086147,0.482569,159027.268519,10129.287037,1955.130658,1.431070
district_7,50198.174717,60887.992415,47.972429,0.313011,125234.332921,11437.024752,1950.638614,1.465099
district_8,39652.894763,52983.805261,30.361766,0.551982,158730.732000,13534.752000,1956.425333,1.457333
district_9,38772.043383,50343.747916,48.448996,-0.313505,100983.590308,12213.361233,1951.126285,1.278267
district_10,60098.906312,84603.221843,45.820451,0.572652,190825.944681,7570.417021,1936.978723,1.878723


## Spatial out-of-sample CatBoost

In [17]:
estimator = CatBoostRegressor()
error_df_soos, col_names_cat, avg_fis_cat, metrics = soos_validation(estimator, structured)
maes, rmses, mapes, r_squareds = metrics

Predicting district 1/13
Predicting district 2/13
Predicting district 3/13
Predicting district 4/13
Predicting district 5/13
Predicting district 6/13
Predicting district 7/13
Predicting district 8/13
Predicting district 9/13
Predicting district 10/13
Predicting district 11/13
Predicting district 12/13
Predicting district 13/13

Weighted metrics:
MAE:  44032
RMSE: 60710
MAPE: 27.24%
R^2:  0.58


In [18]:
error_df_soos.to_csv(PATH+"results/errors_soos_basic.csv")
error_df_soos

,id,lat,long,district,prediction,error
0,71623,40.664950,-79.719952,district_7,150786.728532,35188.271468
1,396546,40.450605,-80.217889,district_1,168956.347357,26043.652643
2,161454,40.477051,-79.777996,district_8,200921.776454,29078.223546
3,461687,40.606085,-79.930633,district_3,671780.867265,-76780.867265
4,136057,40.378484,-80.062464,district_5,447814.315535,150185.684465
...,...,...,...,...,...,...
9549,289011,40.551787,-79.996615,district_1,283985.970741,-8985.970741
9550,250118,40.451706,-79.854802,district_10,119658.450867,341.549133
9551,263392,40.607721,-79.930186,district_3,182546.490634,-87546.490634
9552,459551,40.382627,-80.064891,district_5,294736.867763,-107736.867763


In [19]:
districts = ["district_"+str(i) for i in range(1,14)]
metrics_df = pd.DataFrame(data={"district":districts, "mae":maes, "rmse":rmses, "mapes":mapes, "R^2":r_squareds})
metrics_df = metrics_df.set_index("district")
metrics_df.to_csv(PATH+"results/errors_soos_district_basic.csv")

# add more information about each district to characterize
df_agg = structured.groupby(by="DISTRICT").mean()
df_agg  = df_agg[["SALEPRICE", "LOTAREA", "YEARBLT", "STORIES"]]
metrics_df_agg = pd.concat([metrics_df, df_agg], axis=1)

metrics_df_agg

,mae,rmse,mapes,R^2,SALEPRICE,LOTAREA,YEARBLT,STORIES
district_1,32879.092119,46064.039239,15.941183,0.739438,219888.788503,18311.494577,1959.376356,1.557158
district_2,57489.473713,81481.522775,16.796213,0.743886,347195.029630,28928.464550,1968.440212,1.703175
district_3,44998.750232,68694.874952,18.204345,0.825918,261041.727106,22902.147436,1959.855311,1.500916
district_4,34495.569718,48000.094840,21.812557,0.730342,191448.015873,16155.658009,1956.578644,1.485570
district_5,49345.371365,67181.183912,17.777508,0.696567,279481.883882,13725.758232,1956.907279,1.650347
district_6,27483.245197,37278.238264,21.176568,0.619472,159027.268519,10129.287037,1955.130658,1.431070
district_7,47043.192829,58053.591520,46.703012,0.375482,125234.332921,11437.024752,1950.638614,1.465099
district_8,37653.283651,50777.636483,29.467311,0.588515,158730.732000,13534.752000,1956.425333,1.457333
district_9,38693.094173,46992.341442,50.555751,-0.144444,100983.590308,12213.361233,1951.126285,1.278267
district_10,56643.084897,81402.497325,40.738271,0.604375,190825.944681,7570.417021,1936.978723,1.878723


In [20]:
df_agg.reindex(index=df_agg.index.to_series().str.rsplit('_').str[-1].astype(int).sort_values().index)

,SALEPRICE,LOTAREA,YEARBLT,STORIES
DISTRICT,,,,
district_1,219888.788503,18311.494577,1959.376356,1.557158
district_2,347195.029630,28928.464550,1968.440212,1.703175
district_3,261041.727106,22902.147436,1959.855311,1.500916
district_4,191448.015873,16155.658009,1956.578644,1.485570
district_5,279481.883882,13725.758232,1956.907279,1.650347
district_6,159027.268519,10129.287037,1955.130658,1.431070
district_7,125234.332921,11437.024752,1950.638614,1.465099
district_8,158730.732000,13534.752000,1956.425333,1.457333
district_9,100983.590308,12213.361233,1951.126285,1.278267


## Explore solution

### Coefficients for linear model

In [21]:
coef_df = pd.DataFrame(data={"feature": col_names_linear, "coef": avg_fis_linear})
coef_df.sort_values(by=["coef"], ascending=False).head(15)

,feature,coef
31,STYLEDESC_SEMIDETACHED,29332.452804
8,FULLBATHS,24641.157961
25,STYLEDESC_BUNGALOW,22253.325207
9,HALFBATHS,12908.716811
18,SALEYEAR_STR_2020,11324.227273
30,STYLEDESC_RANCH,9368.712525
23,OWNERDESC_REGULAR-ETUXORETVIR,9236.422071
52,ROOFDESC_TILE.1,8538.544081
44,ROOFDESC_TILE,8538.544081
48,BASEMENTDESC_Part,8121.495582


### Feature importances for CatBoost model

In [22]:
fi_df = pd.DataFrame(data={"feature": col_names_cat, "importance": avg_fis_cat})
fi_df.sort_values(by=["importance"], ascending=False).head(15)

,feature,importance
3,GRADE,30.971280
12,FINISHEDLIVINGAREA,19.663563
0,LOTAREA,7.341299
2,YEARBLT,6.610392
8,FULLBATHS,5.268115
5,CDU,4.761546
6,TOTALROOMS,3.353088
9,HALFBATHS,2.248479
11,BSMTGARAGE,2.174859
10,FIREPLACES,1.892761
